# BYOKG RAG using Neptune Analytics as GraphStore and Vector Store 
This notebook demonstrates a RAG (Retrieval Augmented Generation) system built on top of a Knowledge Graph. In this example, we demonstrate how text embeddings can be used within the BYOKG framework and use a Neptune Analytics graph as the graphstore and the vector store for embeddings. The overall system allows querying a knowledge graph using natural language questions and retrieving relevant information to generate answers.

1. **Graph Store**: Neptune Analytics endpoint for the graph structure and for storing embeddings based on the graph
2. **KG Linker**: Links natural language queries to graph entities and paths
3. **Entity Linker**: Matches entities from question text to graph nodes
6. **Query Engine**: Orchestrates all components to answer questions

#### Setup
If you haven't already, install the toolkit and dependencies in [README.md](../../byokg-rag/README.md).
Let's validate if the package is correctly installed.

In [ ]:
# !pip install https://github.com/awslabs/graphrag-toolkit/archive/refs/tags/v3.8.1.zip#subdirectory=byokg-rag

In [ ]:
from graphrag_toolkit.byokg_rag.graphstore import NeptuneAnalyticsGraphStore

### Graph Store
The `NeptuneAnalyticsGraphStore` class provides an interface to work with the Neptune Analytics graph.
If you already have a NeptuneAnalyticsGraphEndpoint you want to use, simply change the cell below to assign `graph_identifier` to your NeptuneAnalytics graph id. 

If you don't already have a Neptune Graph then you can create one by running the command below from an environment that has the AWS CLI configured with appropriate permissions. Please refer to documentation for more details about [creating a graph](https://docs.aws.amazon.com/neptune-analytics/latest/userguide/create-graph-using-console.html) and [loading data into the graph](https://docs.aws.amazon.com/neptune-analytics/latest/userguide/batch-load.html).

```
aws neptune-graph create-graph --graph-name 'test-kg-with-embedding' --provisioned-memory 128 --public-connectivity --replica-count 0 --vector-search-configuration '{"dimension": 1024}'
```

After running the command you should receive a response that includes the graph id. Change the cell below to assign  `graph_identifier` to the id.

To run the rest of the notebook, you'll need to ensure that the environment has the right IAM permissions to interact with your neptune analytics graph endpoint. Specifically you will need `neptune-graph:ReadDataViaQuery` and `neptune-graph:GetGraph`. You will also need s3 IAM read permissions so that `graphstore.read_from_csv` can access data from `s3://aws-neptune-customer-samples-*/*` and optionally, s3 IAM read and write permissions to your s3 bucket so that embeddings can be saved and loaded from your desired s3 location.

In the rest of the notebook, we
1. Initialize the BYOKG graph store to use a Neptune Analytics Graph
2. Optionally, load an example data from a CSV file for a new graph and get basic statistics
3. Demonstrate using local embedding models and a local vector store how embeddings are generated and used for retrieval and linking.
4. Finally, combine all the steps using the NeptuneAnalyticsGraphStore and BYOKGQueryEngine to combine all the steps into a RAG pipeline and answer a sample question

In [ ]:
region = "us-east-1" #replace with aws region
graph_identifier = "<>" # replace with graph id 

In [ ]:
graph_store = NeptuneAnalyticsGraphStore(graph_identifier=graph_identifier,
                                         region=region)

#### Loading Data

If you ran the command to create a new graph, then uncomment the code cell below to load the new graph with some data. The data we are loading is a KG with information about AWS blog posts on Neptune and Neptune Analytics.

In [ ]:
#graph_store.read_from_csv(s3_path=f"s3://aws-neptune-customer-samples-{region}/sample-datasets/gremlin/KG/")

In [ ]:
# Print graph statistics
number_of_nodes = len(graph_store.nodes())
number_of_edges = len(graph_store.edges())
print(f"The graph has {number_of_nodes} nodes and {number_of_edges} edges.")

In [ ]:
# Print graph schema
import json

schema = graph_store.get_schema()
print(json.dumps(schema, indent=4))


### Node Textual Representation for Embedding and  Vector Index

Now that we have seen the graph schema, we can start to assign which properties will be useful in generating a text representation for each node. Below we create a dictionary where each key is a node label or node type and the corresponding values are the properties to use to represent that node. The final text representation of each node is a stringified json of the property keys and their values for that node as shown in the output of cell below

In [ ]:
node_embedding_text_properties = {
    "organization": ["type", "text"],
    "author": ["name"],
    "title": ["text"],
    "commercial_item": ["type", "text"],
    "tag": ["tag"],
    "location": ["type", "text"],
    "post": ["title"],
    "date": ["type", "text"]
}
node_ids, texts_to_embed = graph_store.get_node_text_for_embedding_input(node_embedding_text_properties)

print(node_ids[:3])
print(texts_to_embed[:3])

#### Local embedding model and index
Once we have texts to embed then we can create an embedding model and use that to generate an embeddings for this node. To illustrate in some details how the embedding model and embedding index, we will use an embedding model that can be run locally via langchain_hugging_face and we will use a local dense index or vector store. The vector store is based on the faiss library.

In [ ]:
from graphrag_toolkit.byokg_rag.indexing import LocalFaissDenseIndex, HuggingFaceEmbedding


embedding_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3",
                                       model_kwargs={"device":"cpu"}, #change to 'cuda' if gpu is available
                                       encode_kwargs={"batch_size": 8},
                                       multi_process=True,
                                       show_progress=True)

create_index_args = {"embedding": embedding_model, "distance_type":"inner_product", "embedding_dim": 1024}

faiss_index = LocalFaissDenseIndex(**create_index_args)
faiss_index.add_with_ids(node_ids, texts_to_embed)

Now let's test retrieval directly from the vector store with a few questions

In [ ]:
input_question = "Did any posts show case a media and entertainment use case?"
response = faiss_index.query(input_question, topk=3)
print(response['hits'])

In [ ]:
input_question = "Did any posts show case a migration use case?"
response = faiss_index.query(input_question, topk=3)
print(response['hits'])

The `EntityLinker` uses query or extracted entities to match against node embeddings saved in the embedding index.

To use this we convert our index to an entity matcher which we then use to initialize an EntityLinker. This will return just node ids in the graph that are relevant to the query

In [ ]:
from graphrag_toolkit.byokg_rag.graph_retrievers import EntityLinker
entity_linker = EntityLinker(retriever=faiss_index.as_entity_matcher())

linked_entities = entity_linker.link([input_question], return_dict=False)
print(linked_entities)

### Neptune Analytics GraphStore Embedding Index

Now that we have validated and examined how text to be embedded is prepared, how the embedding is generated and used for entity linking, let's examine how to put it all together and use managed APIs for embedding generation and Neptune Analytics as the vector store to store the embeddings.

The NeptuneAnalyticsGraphStore class has a convenient `.as_embedding_index` function which can accepts a `graphrag_toolkit.byokg_rag.indexing.Embedding` object and dictionary containing the properties to use to generate the text input to the embeddings. Specifying `load=True` in this function means that graphstore will generate the node text for each node, compute the embeddings and save the embeddings as a vector in the Neptune Analytics graph while `load=False` just directly returns an index object for retrieval. This is useful is the graph already contains embeddings. You can also pass in `embedding_s3_save_location` which is a s3 file location that will be used to load the embeddings from if the already exists, but if the file doesn't exist then it will be created and the computed will also be stored in that s3 file.

In this cell, we use the cohere embedding model on Bedrock which 

In [ ]:
from graphrag_toolkit.byokg_rag.indexing import NeptuneAnalyticsGraphStoreIndex, LLamaIndexBedrockEmbedding
from graphrag_toolkit.byokg_rag.graph_retrievers import EntityLinker

index = graph_store.as_embedding_index(embedding=LLamaIndexBedrockEmbedding(model_name="cohere.embed-english-v3"),
                                       node_embedding_text_props=node_embedding_text_properties,
                                       load=True,
#                                       embedding_s3_save_location='<replace with s3 file name>'
                                      )

entity_linker = EntityLinker(retriever=index.as_entity_matcher())

### BYOKG RAG Pipeline for QA with Neptune Analytics Embedding

Now let's use the `ByoKGQueryEngine` to combine create a question answering pipeline with our graphstore and embedding index for entity linking. To get more details about the different graph retrievers in `ByoKGQueryEngine`, see the `byokg_rag_neptune_analytics_demo.ipynb` notebook

In [ ]:
# set node property for graph store and graph traversal to use to understand and verbalize each node ids
text_repr_prop_for_node = {
    "organization": "text",
    "author": "name",
    "title": "text",
    "commercial_item": "text",
    "tag": "tag",
    "location": "text",
    "post": "title",
}
graph_store.assign_text_repr_prop_for_nodes(text_repr_prop_for_node)

In [ ]:
# set a question to test queries
question = "Who is the author of post on migrating from blazegraph to amazon neptune"

In [ ]:
# create and run query engine

from graphrag_toolkit.byokg_rag.byokg_query_engine import ByoKGQueryEngine

byokg_query_engine = ByoKGQueryEngine(
    graph_store=graph_store,
    entity_linker=entity_linker,
    direct_query_linking=True,
)

retrieved_context = byokg_query_engine.query(question)
answers, response = byokg_query_engine.generate_response(question, "\n".join(retrieved_context))

print(retrieved_context)
print(answers)
print(response)